# Implementing a Deliberative Agent in LangGraph

> **Learning Outcomes:**
> - Understand the concept of a Deliberative Agent
> - Implement a Deliberative Agent in LangGraph
> - Test the Deliberative Agent in a simulated environment

## Introduction

In this lab, we will implement a Deliberative Agent in LangGraph. A Deliberative Agent is an agent that can reason about its actions and make decisions based on its goals and beliefs. The agent first plans its actions before executing them. This is in contrast to a Reactive Agent that reacts to its environment without planning.


### LangGraph Framework

[LangGraph](https://www.google.com/url?sa=E&source=gmail&q=https://python.langchain.com/docs/langgraph) is a framework from Langchain designed for building conversational AI agents with state management. It allows you to create agent workflows as graphs, making complex interactions easier to define and manage.


### The Scenario

We will be creating an LangGraph graph which first plans the actions to complete a task and then executes the actions. The task is finalized by an agent before begin returned to the user. It can be visualized like this:

```mermaid
graph TD
    User --> Planner
    Planner --> Executor
    Executor -- Still Working --> Executor
    Executor -- Tasks Complete --> Finalizer
    Finalizer --> User
```


## Getting Started
Let's start by installing the required libraries and setting the OpenAI API key. The OpenAI API key is required to access the OpenAI models.

Run the following cells to install the required libraries and set the OpenAI API key.


In [ ]:
%pip install --upgrade pip setuptools wheel
%pip install tiktoken --only-binary=:all:

In [ ]:
%pip install -qU \
    langchain==0.3.* \
    langchain_openai==0.3.* \
        langgraph==0.5.*

import os
import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

## Core

### Step 1 - Define the Agent's State

In this step, we will define the state of our Deliberative Agent. The state will include the task, the plan, the current task, the total number of tasks, the results of each task, and the final result.

We will use the `TypedDict` class from the `typing_extensions` module to define the state. This will allow us to specify the types of the state variables.

Here is a simple example of how to define the state of the agent:

```python
class State(TypedDict):
    messages: list[str]
```

In the follow cell, define a `State` class with the following fields:
- `task`: The task that the agent needs to complete.
- `plan`: The plan of action for the task.
- `current_task`: The current task number that the agent is working on.
- `task_count`: The total number of tasks in the plan.
- `results`: A list of results from the completed tasks.
- `final_result`: The final result after all tasks are completed.

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict

# YOUR CODE HERE


This is how LangGraph will pass data between nodes in the graph. As the data moves through the graph, it will be stored in a typed dictionary. Fields can be added or updated as the data moves through the graph.

### Step 2 - Create a Planning Chain

In this step, we will create a planning chain that will generate a plan of action for the agent. The planning chain will use a prompt template to instruct the language model to break down the task into smaller, manageable subtasks.

Here is an example of how to create a chain in LangChain:

```python
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("user", "Create a plan for: {task}"),
])

chain = prompt | llm
chain.invoke({"task": "Build a house!"})
```

<div class="alert alert-info">
    <b>Note:</b> The <code>ChatPromptTemplate</code> class is used to create a prompt template that can be used to interpolate variables into the prompt. Any field from the state can be used in the prompt template.
</div>

In the cell below create a planning chain that generates a plan of action for the agent. Write whatever prompts you think are necessary to instruct the language model to generate a plan of action for the task. Test the chain by invoking it with a task.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel

llm = ChatOpenAI(model="gpt-4.1-mini")


# YOUR CODE HERE


Now that we have a planning chain that generates a plan of action for the agent, we need a bit more information in the output. Our Agent will execute each task iteratively and then finalize the result. It would be nice to know the number of tasks in the plan so that we can track the progress of the agent.

We could parse the output of the planning chain to extract the number of tasks. Or we could use an LLM to extract the number of tasks from the plan. But it is more efficient to modify the planning chain to include the number of tasks in the output.

We can do this using structured output. LangChain allows us to define a schema with Pydantic models to enforce structured output from the planning chain. This ensures that the output from the language model adheres to the structure defined in the schema.

Here is an example of how to define a structured output schema in LangChain:

```python
class SandwhichSchema(BaseModel):
    bread_type: str
    veggies: List[str]
    protiens: List[str]
    sauces: List[str]

llm = llm.with_structured_output(SandwhichSchema)
```

In the following cell, define a Schema to enforce structured output from the planning chain. The schema should have two fields:
- **plan**: A string containing the detailed plan.  
- **task_count**: An integer indicating the number of subtasks generated in the plan.

Update the planning chain to include the structured output schema. Test the chain by invoking it with a task.

In [ ]:
from pydantic import BaseModel

# YOUR CODE HERE


### Step 3 - Create an Execution Chain

Now that we have the planning stage complete, we need to create an execution chain that will execute the plan, step by step. Each call to the execution chain should complete only one task from the plan.

The prompt template should include:

- The original task.
- The plan made by the planner.
- Any previous results.
- The current task number.

Remember that these keys can be accessed from the state dictionary!

In the cell below, create an execution chain that executes one step in the plan. Write whatever prompts you think are necessary to instruct the language model to execute the plan. Test the chain by invoking it with the a state dictionary containing the task, plan, current task number, and results.

In [ ]:
# YOUR CODE HERE


### Step 4 - Create a Finalization Chain

One last step before we can hook everything up! We need to create a finalization chain that will finalize the result of the agent after all tasks are completed. The finalization chain should take the results of all the tasks and combine them into a single final result.

The prompt will look similar to the execution chain, but it should be tasked with combining the results to complete the initial task.

In the cell below, create a finalization chain that finalizes the result of the agent. Write whatever prompts you think are necessary to instruct the language model to finalize the result. Test the chain by invoking it with a state dictionary containing the task and results.

In [ ]:
# YOUR CODE HERE


### Step 5 - Create the Graph

Now it's time to create the graph. The graph will consist of the following nodes:

- **Planner**: The node that generates a plan of action for the agent.
- **Executor**: The node that executes the plan, step by step.
- **Finalizer**: The node that finalizes the result of the agent.

#### Finalizer Node

Each node in the graph needs a function that will be called when the node is invoked. The function should take the state dictionary as input and returns the updated state dictionary. For example:

```python
def house_builder(state: State):
    return { "foundation": foundation_chain.invoke(state).content }
```

<div class="alert alert-info">
    <b>Note:</b> The chains return a Python class. You will likely want to return the <code>content</code> attribute of the chain output.
</div>

The finalizer node should be the easiest to implement. It should take the results of all the tasks and combine them into a single final result. To store the final result it should return a dictionary with the key `final_result`.



In the cell below, define the function for the finalizer node.

In [ ]:
# YOUR CODE HERE


#### Planner Node
Next, we'll define the function for the planner node. The planner node should generate a plan of action for the agent. 

Remember that our planning chain outputs a structured response with the plan and the number of tasks. We can use this information to update the state dictionary. We can also use this node to set the initial values for the `current_task` and `results` fields in the state dictionary.

In the cell below, define the function for the planner node. It should return a state with all of the keys defined.

In [ ]:
# YOUR CODE HERE


#### Executor Node

Our last node is the executor node. This node should increment the `current_task` field--you don't need an LLM to do this! It should also execute the current task in the plan--you do need an LLM for this!

In addition to incrementing the `current_task`, the node should return an updated list of results. This means you will need to append the result of the current task to the list of results.

In the cell below, define the function for the executor node. Make sure that all of the other fields in the state dictionary are preserved.

In [ ]:
# YOUR CODE HERE


#### A Conditional Edge

The executor node will have two outgoing edges: one for when the agent is still working and one for when the agent has completed all the tasks. The edge for when the agent is still working should loop back to the executor node. The edge for when the agent has completed all the tasks should go to the finalizer node.

Conditional edges in LangGraph are defined using... functions! Just like a node, the conditional edge function takes the state dictionary as input. However, the function returns only a string indicating the name of the next node to go to. For example:

```python
def is_house_done(state: State):
    # Check if the agent has completed all the tasks
    for key, value in state.items():
        if value != "done":
            return "build_house"
    return "house_done"
```

In the cell below, define the function for the conditional edge between the executor and finalizer nodes. The function should return `done` if the agent has completed all the tasks, and `next_agent` otherwise. You can use the `current_task` and `task_count` fields in the state dictionary to determine if the agent has completed all the tasks.

In [ ]:
# YOUR CODE HERE


#### Create the Graph
With all our functions specified, we can now create the graph.

In the cell below, is an example of how to create a graph in LangGraph. Run the cell to create the graph and visualize it.


In [ ]:
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display

# Fake state
class HomeState(TypedDict):
    pass

# Fake nodes
house_designer = lambda state: None
house_builder = lambda state: None
realtor = lambda state: None
is_house_done = lambda state: None

builder = StateGraph(HomeState)
builder.add_node("house_designer", house_designer)
builder.add_node("house_builder", house_builder)
builder.add_node("realtor", realtor)
builder.add_edge(START, "house_designer")
builder.add_edge("house_designer", "house_builder")
builder.add_conditional_edges("house_builder", is_house_done, {"house_done": "realtor", "next_agent": "house_builder"})

graph = builder.compile()


# Display the graph
display(Image(graph.get_graph().draw_mermaid_png()))


In the cell below, create the graph for our agent. Display the graph to ensure it is correct.

It should look something like this:

```mermaid
graph TD

    start(["\_\_start\_\_"]) --> planner(planner)
    planner --> executor(executor)
    executor -- next_task --> executor
    executor -- done --> finalizer(finalizer)
    finalizer --> result(["\_\_end\_\_"])
```

In [ ]:
# YOUR CODE HERE


## Step 6 - Test the Agent

Now that we have created the graph, it's time to test our work! In the cell below, is code that will run the agent and output the results.

If your graph is named `graph`, this will work out of the box. If you named your graph something else, you will need to update the code below.

In [ ]:
from IPython.display import Markdown

total_tasks = 0

TASK = "Write a corporate policy on Generative AI usage."

for chunk in graph.stream({"task": TASK }, stream_mode="updates"):
    node = next(iter(chunk)) 
    if node == "finalizer":
        display(Markdown(f"**Final Result**\n {chunk[node]['final_result']}\n\n"))
    elif node == "planner":
        display(Markdown(f"**{node}**\n{chunk[node]['plan']}\n\n"))
        total_tasks = chunk[node].get('task_count', 0)
    elif node == "executor":
        current_task_num = chunk[node].get('current_task', 'N/A')
        display(Markdown(f"**{node}**\n```{current_task_num} / {total_tasks}```\n\n"))

Congratulations! You have successfully implemented a Deliberative Agent in LangGraph. Feel free to experiment with different tasks and prompts to see how the agent performs.

## Bonus Challenge 1 - Iterative Tasks

Right now, our graph expects the agent to complete each task in a single step. This is not very realistic. In reality, each task may require multiple steps to complete. For example, laying the foundation of a house may require multiple steps, such as digging the foundation, pouring the concrete, and letting it set.

Update the graph to allow for multiple steps in each task. You will need to update the executor node to handle multiple steps. You may want an extra node that reviews the results of each task before moving on to the next one.

In [ ]:
# YOUR CODE HERE

## Bonus Challenge 2 - Error Handling

Our agent is very optimistic. It assumes that everything will go according to plan. In reality, things can go wrong. For example, the agent may encounter an unexpected problem while executing a task. Update the graph to handle errors. You may want to add a node that retries the task if it fails.

In [ ]:
# YOUR CODE HERE